In [ ]:
import backtrader as bt
import pandas as pd
import numpy as np


TRADING_YEAR = 252

class StarterStrategy(bt.Strategy):
    params = (
        ('short_term', 16),
        ('long_term', 64),
        ('target_risk', 0.12),
        ('stop_loss_gap', 0.5),
        ('starting_capital', 1000),
        ('shorts', True),
    )

    def __init__(self):
        self.sma1 = bt.indicators.SMA(period=self.params.short_term)
        self.sma2 = bt.indicators.SMA(period=self.params.long_term)
        self.std = bt.indicators.StdDev(period=TRADING_YEAR, movav=bt.indicators.MovAv.Simple)
        self.history = []

    def next(self):
        close = self.data.close[0]
        item = self.StarterStrategyItem(close=close)
        self.history.append(item)
        item.sma1, item.sma2, item.std = self.get_smas_std()
        item.last_exit_direction = 0 if len(self.history) == 1 else self.history[-2].last_exit_direction

        if any(np.isnan([item.sma1, item.sma2, item.std])):
            item.cash = self.params.starting_capital if len(self.history) == 1 else self.history[-2].cash
            return

        direction = 1 if item.sma1 > item.sma2 else -1
        new_stop = self.get_stop_price(close, direction)

        item.position = self.history[-2].position
        item.cash = self.history[-2].cash
        item.stop = self.history[-2].stop

        if item.last_exit_direction == -direction:
            item.last_exit_direction = 0

        if direction == 1:
            if item.position > 0:
                item.stop = max(item.stop, new_stop)
                if close < item.stop:
                    item.cash += item.position * close
                    item.position = 0
                    item.stop_status = 1
                    item.last_exit_direction = 1
            else:
                if item.position < 0:
                    item.cash += item.position * close
                if item.last_exit_direction != 1:
                    item.position = self.get_position_size(close)

        elif direction == -1:
            if item.position < 0:
                item.stop = min(item.stop, new_stop)
                if close > item.stop:
                    item.cash += item.position * close
                    item.position = 0
                    item.stop_status = 1
                    item.last_exit_direction = -1
            else:
                if item.position > 0:
                    item.cash += item.position * close
                if self.params.shorts and item.last_exit_direction != -1:
                    item.position = -self.get_position_size(close)

    def get_smas_std(self):
        close = np.array([item.close for item in self.history])
        sma1 = np.nan if len(close) < self.params.short_term else np.mean(close[-self.params.short_term:])
        sma2 = np.nan if len(close) < self.params.long_term else np.mean(close[-self.params.long_term:])
        std = np.nan if len(close) <= TRADING_YEAR else np.std(np.diff(np.log(close[-TRADING_YEAR:]))) / np.sqrt(TRADING_YEAR)
        return sma1, sma2, std

    def get_stop_price(self, price, direction):
        if direction == 1:
            return price * (1 - self.params.stop_loss_gap * self.std)
        return price * (1 + self.params.stop_loss_gap * self.std)

    def get_position_size(self, price):
        exposure = self.params.target_risk * self.history[-2].cash / self.history[-2].std
        shares = np.floor(exposure / price)
        if shares * price > self.history[-2].cash:
            return np.floor(self.history[-2].cash / price)
        return shares

class StarterStrategyItem:
    def __init__(self, close):
        self.close = close
        self.sma1 = np.nan
        self.sma2 = np.nan
        self.std = np.nan
        self.cash = 0
        self.position = 0
        self.stop = np.nan
        self.stop_status = np.nan
        self.last_exit_direction = 0

# Load data
data = pd.read_csv("historical_forex_data20.csv")
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)

# Setup backtesting
cerebro = bt.Cerebro()
cerebro.broker.set_cash(10000)

# Add data
datafeed = bt.feeds.PandasData(dataname=data)
cerebro.adddata(datafeed)

# Add strategy
cerebro.addstrategy(StarterStrategy)

# Run backtest
cerebro.run()

# Plot results
cerebro.plot()
